In [1]:
import json
import torch
from transformers.generation import GenerationConfig
from torch.utils.data import Dataset, DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer
import os
import re

In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [7]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive')
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NLP_main_project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
DATA_PATH = GOOGLE_DRIVE_PATH
print(os.listdir(DATA_PATH))
file_name = 'train.json'
file_path = os.path.join(DATA_PATH, "data",file_name)
print(file_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['选题二 基于大语言模型的思维链推理.docx', '.DS_Store', 'cot.ipynb', '.gitignore', '.git', '.locks', '.ipynb_checkpoints', 'data']
['选题二 基于大语言模型的思维链推理.docx', '.DS_Store', 'cot.ipynb', '.gitignore', '.git', '.locks', '.ipynb_checkpoints', 'data']
drive/My Drive/NLP_main_project/data/train.json


In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
DATA_PATH = 'data'
file_name = 'train.json'
file_path = os.path.join(DATA_PATH, file_name)

model = LlamaForCausalLM.from_pretrained(
    'huggyllama/llama-7b',
    # load_in_8bit=True,
    device_map={"": 0},
)
tokenizer = LlamaTokenizer.from_pretrained('huggyllama/llama-7b')
# tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
# tokenizer.padding_side = "left"  # Allow batched inference

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [50]:
import json
import torch
from torch.utils.data import Dataset, DataLoader

# 定义自定义数据集类
class QuestionDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['question']
        options = item['options']
        combined_question = question
        for option in options:
            combined_question += f"\n{option}"
        answer = item['correct']
        return combined_question, answer



# 读取数据集
with open(file_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# 创建数据集
data = data[:50]
question_dataset = QuestionDataset(data)
# 创建数据加载器
data_loader = DataLoader(question_dataset, batch_size=1, shuffle=True)

counter = 0
for questions, answers in data_loader:
    print(f"Question: {questions[0]}")
    print(f"Correct answer: {answers[0]}")
    counter += 1
    if counter >= 10:  # 只读取前10个问题和答案
        break

Question: In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?
A)4 and 1
B)1 and 5
C)5 and 1
D)3 and 5
E)5 and 3
Correct answer: C
Question: The speed at which a man can row a boat in still water is 25 kmph. If he rows downstream, where the speed of current is 11 kmph, what time will he take to cover 80 metres?
A)18 seconds
B)27 seconds
C)26 seconds
D)12 seconds
E)8 seconds
Correct answer: E
Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?
A)21
B)21.5
C)22
D)22.5
E)23
Correct answer: E
Question: For all numbers p and q, the operation @ is defined by p@q = p^2 - pq. If xy ≠ 0, then which of the following can be equal to zero?
I. x@y
II. (xy)@y
III. x@(x + y)
A)II
B)I and II
C)I and III
D)

In [43]:
def decode(outputs, tokenizer, raw_text_len):
    """
    Decodes the model outputs using the provided tokenizer and processes the decoded text.

    Args:
        outputs (torch.Tensor): The model outputs.
        tokenizer (Tokenizer): The tokenizer object used for decoding.
        raw_text_len (int): Length of the raw text.

    Returns:
        str: Processed decoded text.
    """
    # Decode the model outputs using the tokenizer
    decoded_text = tokenizer.decode(outputs[0][raw_text_len:])

    # Process the decoded text
    res = decoded_text[raw_text_len:]
    res = res.split("</s>")[0]
    res = res.split("<|endoftext|>")[0]       # Get the first part before an empty string (likely a typo, consider fixing)
    res = res.split("\n\n\n")[0]              # Get the first part before multiple consecutive newlines
    res = res.split("Q:")[0]                  # Get the first part before "Q:" (likely indicating a new question)
    res = res.split("Question:")[0]           # Get the first part before "Question:" (likely indicating a new question)

    return res

def inference(model, tokenizer, question, max_new_tokens):
    """
    Generates text based on a given question using the provided model and tokenizer.

    Args:
        model (PreTrainedModel): The pre-trained model for inference.
        tokenizer (Tokenizer): The tokenizer object used for tokenization.
        question (str): The input question for inference.
        max_new_tokens (int): Maximum number of tokens to generate.

    Returns:
        str: Generated text based on the input question.
    """
    # Tokenize the input question
    input_ids = tokenizer.encode(question, return_tensors="pt")
    input_ids = input_ids.to(model.device)

    # Generate text based on the input question
    outputs = model.generate(input_ids, max_new_tokens=max_new_tokens)

    # Decode and process the generated text
    generated_text = decode(outputs, tokenizer, len(input_ids))

    return generated_text

In [44]:
# 准确率计算函数
def calculate_accuracy(predictions, references):
    correct = 0
    total = len(references)
    for pred, ref in zip(predictions, references):
        if pred == ref:
            correct += 1
    return correct / total

### single path prediction

In [ ]:
predictions = []
prompt_1 = "Let's think step by step."
prompt_2 = "Therefore, among A through E, the lable of the answer is "
for i, (questions, answers) in enumerate(data_loader):
    questions = list(questions)
    answers = list(answers)
    for question, answer in zip(questions, answers):
        input_1 = f"Q: {question}. {prompt_1}"
        output_1 = inference(model, tokenizer, input_1, 256)
        print(f"output1: {output_1}")

        input_2 = f"{input_1} {output_1} {prompt_2}"
        output_2 = inference(model, tokenizer, input_2, 10)
        output_2 = output_2.replace(input_2, "")
        print(f"output2: {output_2}")

        # 提取生成文本中的第一个大写字母作为预测答案
        prediction = re.findall(r'\b[A-E]\b', output_2)[-1]
        if prediction:
            predictions.append(prediction)
            print(f"Predicted Answer: {prediction}")
        else:
            predictions.append(None)  # 如果没有找到大写字母，则预测为空
            print("Predicted Answer: None")

### predict using multipath and vote

In [46]:
def multi_infer_and_vote(model, tokenizer, question, first_prompts, second_prompts):
    """
    infer the answer by voting from len(first_prompts) inference paths, the ith path using first_prompts[i] and second_prompts[i]
    the voting is based on the majority vote from thoes pathes.

    Args:
        model (PreTrainedModel): The pre-trained model for inference.
        tokenizer (Tokenizer): The tokenizer object used for tokenization.
        question (str): The input question for inference.
        first_prompts (list): The first prompts for inference.
        second_prompts (list): The second prompts for inference.
        max_new_tokens (int): Maximum number of tokens to generate.

    Returns:
        selected letter: the voted answer.
    """
    assert len(first_prompts) == len(second_prompts)
    votes = []
    for i in range(len(first_prompts)):
        first_prompt = first_prompts[i]
        second_prompt = second_prompts[i]
        # generate first text
        first_input = f"Q: {question}.{first_prompt}"
        first_output = inference(model, tokenizer, first_input, 350)
        print(f"first_output with prompt{i}: {first_output}")
        # generate second text
        second_input = first_output + second_prompt
        second_output = inference(model, tokenizer, second_input, 10)
        second_output = second_output.replace("second_input","")
        print(f"second_output with prompt{i}: {second_output}")
        # get answer and vote
        ans = re.findall(r'\b[A-E]\b', second_output)[-1]
        if ans:
            votes.append(ans)

    if not votes:
        return None

    return max(set(votes), key=votes.count)

In [51]:
predictions = []
first_prompts = ["Let's think step by step.", "Let's use logical reasoning to solve this.", "The mathematical formula to solve this is: "]
second_prompts = ["Therefore, the answer is", "Therefore, the answer is", "Therefore, among A through E, the answer is"]
true_answer = []
for i, (questions, answers) in enumerate(data_loader):
    questions = list(questions)
    answers = list(answers)
    for question, answer in zip(questions, answers):
        print(f"{i} Answer: {answer}")
        true_answer.append(answer)
        prediction = multi_infer_and_vote(model, tokenizer, question, first_prompts, second_prompts)
        if prediction:
            predictions.append(prediction)
            print(f"Predicted Answer: {prediction}")
        else:
            predictions.append(None)  # 如果没有找到大写字母，则预测为空
            print("Predicted Answer: None")

0 Answer: E
first_output with prompt0: : The speed at which a man can row a boat in still water is 25 kmph. If he rows downstream, where the speed of current is 11 kmph, what time will he take to cover 80 metres?
A)18 seconds
B)27 seconds
C)26 seconds
D)12 seconds
E)8 seconds.Let's think step by step.
1. The speed at which a man can row a boat in still water is 25 kmph.
2. If he rows downstream, where the speed of current is 11 kmph, what time will he take to cover 80 metres?
3. The speed of current is 11 kmph.
4. The distance covered by the man in 1 minute is 80 metres.
5. The distance covered by the man in 1 minute is 80 metres. (1)
6. The distance covered by the man in 1 minute is 80 metres. (2)
7. The distance covered by the man in 1 minute is 80 metres. (3)
8. The distance covered by the man in 1 minute is 80 metres. (4)
9. The distance covered by the man in 1 minute is 80 metres. (5)
10. The distance covered by the man in 1 minute is 80 metres. (6)
11. The distance covered by the

In [59]:
print(len(predictions))
print(len(true_answer))
# 计算准确率
accuracy = calculate_accuracy(predictions, true_answer)
print(f"\nAccuracy: {accuracy * 100:.2f}%")

5
5

Accuracy: 20.00%


In [62]:
# 打印一些样本数据
print("\nSample Outputs:")
for a, p in zip(true_answer[:5], predictions[:5]):
    print(f"True Answer: {a}")
    print(f"Predicted Answer: {p}")


Sample Outputs:
True Answer: E
Predicted Answer: A
True Answer: A
Predicted Answer: A
True Answer: E
Predicted Answer: A
True Answer: C
Predicted Answer: B
True Answer: B
Predicted Answer: C
